In [1]:
import numpy as np
import numpy.random as rnd
import time
import os

import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#### Все места, где нужно дописать код отмечены TODO.

## Считывание и подготовка данных.

In [2]:
os.chdir('Data')

In [3]:
# Считываем текстовые данные: все файлы должны лежать в одной папке data. 
# Проверьте, что у вас все хорошо с кодировками и текст нормально считывается.
data = ""

for fname in os.listdir("data"):
    with open("data/"+fname) as fin:
        text = fin.read().decode('cp1251')
        data += text

In [4]:
print data[:200]

---------------------------------------------


 Устав патрульно-постовой службы милиции общественной безопасности Российской Федерации

 Утвержден приказом Министра внутренних дел Российской Фед


In [5]:
# Для дальнейшей работы нам нужно текст перевести в числовой формат.
chars = list(set(data))
VOCAB_SIZE = len(chars)

char_to_id = { ch:id for id,ch in enumerate(chars) }
id_to_char = { id:ch for id,ch in enumerate(chars) }
data_ids = [char_to_id[ch] for ch in data]

## Вспомогательные функции

In [6]:
# Необходимые константы
NUM_EPOCHS = 10
NUM_BATCHES = 1000
BATCH_SIZE = 100
SEQ_LEN = 20
LEARNING_RATE = 0.01
GRAD_CLIP = 100

В процессе обучения мы будем для каждого символа входной последовательности предсказывать следующий символ. Таким образом на вход сети мы будем подавать последовательности длины SEQ_LEN и получать на выходе последовательности той же длины, но со сдвигом на один символ.

In [7]:
def generate_random_batch(source):
    """Функция, которая генерирует batch из BATCH_SIZE случайных подстрок текста source. 
    Каждая подстрока должна иметь длину SEQ_LEN.
    
    source - массив целых чисел - номеров символов в тексте (пример - data_ids)
    
    Вернуть нужно кортеж (X,y), где
    X - матрица, в которой каждая строка - подстрока длины SEQ_LEN (подается на вход сети)
    y - матрица, в которой каждая строка - подстрока длины SEQ_LEN, (ожидается на выходе сети)
    Таким образом, каждая строка в y должна соответсвовать строке в X со сдвигом на один символ вправо.
    Например, если X[0]='hell', то y[0]='ello'
    
    Убедитесь, что вы генерируете X и y, которые правильно соответствуют друг другу.
    Также убедитесь, что ваша функция не вылезает за край текста (самое начало или конец текста).
    """
    
    # TODO
    
    return X_batch, y_batch

In [9]:
a,b = generate_random_batch(data_ids)
print ''.join(id_to_char[id] for id in a[0,:])
print ''.join(id_to_char[id] for id in b[0,:])

органов указанного ф
рганов указанного фе


В процессе тестирования мы будем предсказывать следующий символ по SEQ_LEN предыдущих. 
Генерировать очередной символ в тестовой посдедовательнсоти можно разными способами:
1. max_sample_fn: брать символ с максимальной вероятностью
2. proportional_sample_fn: генерировать символ пропорционально вероятности
3. alpha_sample_fn: генерировать символ пропорционально вероятности со следующей предобраоткой: 
    logprobs/alpha, где alpha - "жадность" из (0,1] - чем меньше, тем ближе генерация к выбору максимума
    после взятия экспоненты такие вероятности нужно перенормировать

Для устойчивости вычислений наша сеть будет выдавать не вероятности, а их логарифмы, поэтому не забывайте в нужных местах брать от них exp.

In [16]:
def max_sample_fn(logprobs):
    return np.argmax(logprobs) 

def proportional_sample_fn(logprobs):
    # TODO

def alpha_sample_fn(logprobs, alpha):
    # TODO

def generate_seed():
    """Функция выбирает случайное начало поседовательности из data, 
    которую мы потом можем продолжать с помощью нейросети.
    """
    start = np.random.randint(0,len(data)-SEQ_LEN)
    seed_phrase = data[start:start+SEQ_LEN]
    return seed_phrase

def generate_sample(logprobs_fn,sample_fn,seed_phrase,N=100):
    """Функция генерирует случайный текст при помощи нейросети и печатает его
    
    logprobs_fn - функция, которая по входной последовательности длины SEQ_LEN 
        предсказывает логарифмы вероятностей посдледующего символа (см. функцию train)
    sample_fn - функция, выбирающая следующий символ одним из способов, описанных выше
    seed_phrase - начальная фраза, с которой мы начинаем генерировать
    N - размер генерируемого текста
    
    """
    
    # TODO
    
    print(random_snippet)

In [11]:
# Технические вещи

# Вспомогательная функция для запаковки результата обучения 
def pack(err, network, logprobs_fn):
    return {'err':err, 
        'network':network,
        'logprobs_fn':logprobs_fn
           } 

# numerically stable log-softmax with crossentropy
def logsoftmax(x):
    xdev = x-x.max(2,keepdim=True)[0]
    lsm = xdev - torch.exp(xdev).sum(dim=2, keepdim=True).log()
    return lsm

def lsmCE(x,y):
    return -torch.clamp(x,-20,0).gather(2, y.unsqueeze(2)).squeeze().mean()

## Нейронная сеть

In [12]:
class Net(nn.Module):
    """Класс задает простейшую рекуррентную сеть, которая принимает на вход батч размера [BATCH_SIZE, SEQ_LEN] 
    и применяет к нему следующие преобразования:
    
    1. Embedding для перевода кодировки символов в нормальное представление: VOCAB_SIZE -> emb_size
    2. Рекуррентный слой c n_hidden элементов на скрытом слое.
    3. Полносвязный слой n_hidden -> VOCAB_SIZE с logsoftmax в качестве нелинейности.
    
    В итоге на выход сеть должна возвращать ответ размера [BATCH_SIZE, SEQ_LEN, VOCAB_SIZE] 
    
    * Обратите внимание на параметр batch_first у рекуррентного слоя.
    """

    
    def __init__(self, emb_size = 40, n_hidden = 100):
        # TODO

    def forward(self, text):
        # TODO

In [13]:
def train(data_ids, emb_size, n_hidden, show = False):
    """Функция обучает нейросеть по данным data_ids
    Следует обратить внимание на следующее:
    1. Сеть будем учить NUM_EPOCHS эпох, в каждой из которых будет NUM_BATCHES батчей
    2. Для того, чтобы следить за процессом обучения будем считать средний loss 
        на всех батчах в эпохе и сохранять его в массив err. Также будем генерировать 
        последовательности из случайных seeds после каждой эпохи, для этого нужна будет функция logprobs_fn,
        которая по входу х размера [1, SEQ_LEN] будет выдавать вектор логарифмов вероятностей 
        для последующего символа размера [1, VOCAB_SIZE]. 
        Например, если x='hell', то нас интересует каким будет символ после второго l. 
    3. Так как мы вместо softmax используем logsoftmax, то в качестве loss для сети нужно использовать lsmCE
    4. Перед тем, как делать шаг по градиенту, шрадиент нужно ограничивать по норме значением GRAD_CLIP
    
    * Если вы используете GPU, то не забудьте все данные и саму сеть перенести на GPU.
    """
    err=np.zeros(NUM_EPOCHS)

    print("Building network ...")
    # Строим сеть и переносим ее на cuda, если нужно
    # TODO
    print("The network has {} params".format(sum([x.data.numel() for x in net.parameters()])))
    
    # Задаем оптимизатор, рекомендуется использовать adam
    # TODO
        
    def logprobs_fn(snippetIdx):
        # TODO
        
    print("Training ...")
    for epoch in xrange(NUM_EPOCHS):
        start_time = time.time()
        for batch in xrange(NUM_BATCHES):
            # TODO

        if show:
            seed = generate_seed()
            print "Seed: '{}'".format(seed.encode('utf-8'))
            print "Max sample:"
            generate_sample(logprobs_fn, max_sample_fn, seed)
            print "Proportional sample:", 
            generate_sample(logprobs_fn, proportional_sample_fn, seed)
        print("Epoch {} \t loss = {:.4f} \t time = {:.2f}s".
                      format(epoch, err[epoch], time.time() - start_time))
             
    return pack(err, net, logprobs_fn)

Перед тем, как запускать обучение с большим числом итераций и длинными последовательностями, попробуйте запустить его на десяток итераций с последовательнсотямит по 5 символов и проверьте, что у вас генерируются какие-то вменяемые слоги. При этом достатояно использовать довольно маленькую сеть.

In [14]:
model = train(data_ids, 40, 200, show = False)

Building network ...
The network has 71182 params
Training ...
Epoch 0 	 loss = 2.4505 	 time = 4.84s
Epoch 1 	 loss = 1.7952 	 time = 4.64s
Epoch 2 	 loss = 1.5806 	 time = 4.62s
Epoch 3 	 loss = 1.4649 	 time = 4.63s
Epoch 4 	 loss = 1.3969 	 time = 4.64s
Epoch 5 	 loss = 1.3532 	 time = 4.62s
Epoch 6 	 loss = 1.3178 	 time = 4.63s
Epoch 7 	 loss = 1.2906 	 time = 4.62s
Epoch 8 	 loss = 1.2705 	 time = 4.65s
Epoch 9 	 loss = 1.2560 	 time = 4.68s


## Посмотрим что из этого вышло

In [17]:
seed = u"Каждый человек должен"
alpha = 0.5
sampling_fun = lambda x: alpha_sample_fn(x, alpha)
result_length = 300

generate_sample(model['logprobs_fn'],sampling_fun,seed,result_length)

Каждый человек должен вознай или в соответствии с порядке, нех соответствии с принятия наследниками или подлежат исполнения потожный судебной статьи 19.5, статьями 19.20 настоящей статьи 118 настоящего Кодекса и лицами совершение возника

 1. Права и правилами собственника определение осуществляющие порядке, после ино


In [18]:
seed = u"В случае неповиновения"
alpha = 0.5
sampling_fun = lambda x: alpha_sample_fn(x, alpha)
result_length = 300

generate_sample(model['logprobs_fn'],sampling_fun,seed,result_length)

В случае неповиновения

 1. Проведение товаров и принятия в соответствии с принятия в порядке, об административного штрафа на путем о принятия при право граждан от двадцати до действия об административного правонарушении обращение предусмотрено должностных лиц – от десяти до трех действии содержащей судом или соответст


## Дополнительные пункты

1. Обучение более сложной модели и контроль переобучения. Попробуйте подобрать хорошую модель RNN для данной задачи. Для этого  проанализируйте качество работы модели в зависимости от ее размеров, попробуйте использовать многослойную сеть. Также нужно проконтролировать переобучение моделей. Для этого можно выделить тестовый кусок из текста и смотреть на то, как меняется loss на нем в процессе обучения. Если на графиках видно переобучение, то стоит добавить dropout в модель (обычный dropout до, между и после рекуррентных слоев). 
2. LSTM и GRU архитектуры. Вместо обычной RNN попробуйте LSTM и GRU архитектуры и сравните получающиеся результаты для моделей нескольких разных размеров. Также сравните модели на данных с разной SEQ_LEN. 
4. Визуализация. Попробуйте провизуализировать результаты. Например, можно смотреть на то, какие буквы модель хорошо предсказывает, а в каких сильно не уверена. Это покажет что именно выучила модель лучше всего. Также можно попробовать смотреть на активации разных скрытых нейронов при прочтении текста (как у Андрея Карпатого).
5. Более сложные данные. Попробуйте обучить модель на более структурированных данных, например коде. Используйте LSTM и GRU сети, они хорошо улавливают структуру в данных. Проанализируйте результаты: выделите нейроны, активации которых "отвечают" за структуру в данных. Этот пункт, пожалуй, стоит пробовать только если у вас есть нормальный GPU.
6. Продвинутый дропаут. Запрограммировать RNN/LSTM с продвинутым дропаутом из (одним из 3, обсужденных на лекции). Сравнить с обычным вариантом дропаута по нерекуррентным связям.